In [2]:
import os
%pwd

'c:\\Users\\Public\\Documents\\END-TO-END-PROJECT\\DL-PROJECT\\Chicken-diseases-classification\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Public\\Documents\\END-TO-END-PROJECT\\DL-PROJECT\\Chicken-diseases-classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from chicken_diseases_prediction.constants import *
from chicken_diseases_prediction.utils.common import  read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [7]:
import os
import urllib.request as request
import zipfile
from chicken_diseases_prediction import logger
from chicken_diseases_prediction.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-01 11:55:33,756: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-01 11:55:33,758: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-01 11:55:33,759: INFO: common: created directory at: artifacts]
[2023-10-01 11:55:33,761: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-01 11:55:43,211: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
X-GUploader-UploadID: ADPycdsysoMrY-5zwrf7oIXTLcFmaFltaI1664mu3W8IlY8xr14lvD6D54TyRNYIlZIPk7mOlUJGZciSDTBmyHLvfOdBHg
X-Content-Type-Options: nosniff
Content-Type: application/x-zip-compressed
Content-Disposition: attachment; filename="Chicken-fecal-images.zip"; filename*=UTF-8''Chicken-fecal-images.zip
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: false
Access-Control-Allow-Headers: Accept, Accept-Language, Authorization, Cache-Control, Content-Disposition, Content-Encoding, Content-Language, Content-Length, Content-MD5, Content-